# Python libraries and databases

<div style="background: #CCFFCC; padding: 0.5em"><h1>About this notebook and its embedded assignments</h1>
<p>This notebook moves somewhat quickly through some challenging topics. It introduces new things, and also has four exercises embedded at various points. You don't need to understand and learn everything at once. An important step in ongoing learning with material like this is becoming adept at picking up clues from context, exploring on your own, modifying code as a path to understanding it, and deferring some things so that you can come back to them later.</p>
    <p>The four exercises are not meant to be especially challenging on their own. They do ask you to fill in code in particular cells. Please execute <b>all</b> the cells in the notebook, <b>including</b> the cells with your exercises, so that the output is visible below each cell (whenever there is output). Then save that version of the notebook and upload it to Canvas. </p></div>

Earlier we learned some basic features of the Python language. Although there is more to learn about fundamental features of Python, for many particular tasks a lot of the value of a programming language comes from using libraries of code that other people have written to do specialized things. Python has an especially rich ecosystem of libraries across a wide range of tasks, from text processing to web programming to statistics and machine learning. 

To use a library in our own code, we have to "import" it, that is, tell the Python interpreter that we are going to use it. 

Good common practice is to put all `import` statements at the top of a file, but for pedagogical purposes in this notebook they will be imported throughout, one by one, as needed.

Some libraries are built into Python, and many additional libraries are included with the Anaconda distribution of Python. If a library isn't already installed to be available for import on the local system, it will first have to be installed, perhaps using the Anaconda Navigator environment manager, or using "conda" or "pip" on the command line. For this exercise we will be using libraries that should be included with Anaconda already, so I hope you don't have to worry for now about "conda" or "pip". If you keep working with Python, at some point you will need them.

<hr/>

<h2 style="background: #FFCCCC; padding: 1em">Setup information <b>if</b> you are using Google Colab</h2>

Google Colab starts up a new virtual machine for you when you run a notebook. It has its own RAM and its own local storage. Although you can use this storage space and upload and download files from it, the storage itself disappears when the notebook stops and the virtual machine shuts down. 

To get longer-term storage, you can connect Colab with Google Drive, and get access to files you have stored there, or store new files there. If the code in the next cell weren't already there for you, you could click on "Mount drive" in the Files tab of the Table of Contents to the left, and it would fill in that code for you.

When you run the next cell, you'll import `drive` from the Google Colab module, and then use the `mount` method to map your Google Drive storage to a path in in the filesystem of the virtual machine that your Colab instance is running for you. There is an interactive step that requires you to click through, give your authorization so that your Colab instance can access your Google Drive, and then paste an authorization code so that your Colab instance can authenticate itself.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Uploading

I created a folder called `iph430` in my Google Drive account, and then uploaded the `bookcrossing.sqlite` to that folder. You can choose a different structure or name if you like, and will have to adjust your paths accordingly. 

In [ ]:
### Here's another option that would upload a file to a temporary space (not on Google Drive) where your notebook is running. 
### It's much slower, in my experience, but it's an option. You would need to uncomment the next two lines to get them to run.
### But I would advise ignoring these cells and just uploading through the Google Drive interface.

# from google.colab import files
# files.upload()

In [ ]:
### If you followed that `files.upload()` option and wanted to move the file to Google Drive, you could 
### uncomment and run the last two lines in this cell.
### (A line starting with ! is interpreted as a shell command rather than Python code.)

# !mkdir -p /content/drive/My\ Drive/iph430/bookcrossing.sqlite3
# !mv bookcrossing.sqlite3 /content/drive/My\ Drive/iph430/bookcrossing.sqlite3

<h2 style="background: #FFCCCC; padding: 1em">[End of special setup for Google Colab]</h2>
<hr/>

# Connecting to a SQLite database

Here is how we import a library. After running the next cell, nothing will have changed except the capacity of our environment, which will now understand references to the `sqlite` library and its contents.

In [1]:
import sqlite3

Now if we evaluate the name `sqlite3`, we will see that it's a "module" object of some kind. That's basically another way of saying "library".

In [2]:
sqlite3

<module 'sqlite3' from '/home/knoxdw/anaconda3/lib/python3.9/sqlite3/__init__.py'>

If you are running locally and didn't need to do the Google Colab setup, find the path to your bookcrossing SQLite data file, or move or upload a copy of the file so that it is in the same place where you are running this Jupyter notebook.

Modify the path in the next cell as needed, and then when you evaluate the cell, you should have an open connection to your SQLite database.

The connection is like hooking up a hose to a pipe without turning on the valve. It prepares a connection, but it doesn't yet make any data flow through the pipe.

In [3]:
# A path on Google Drive may look like this, adjusting folder names as needed:

# conn = sqlite3.connect("/content/drive/My Drive/iph430/bookcrossing.sqlite3")

# A local path may be similar, starting with '/Users/...', 
# or could just be a relative path to the filename, if your notebook 
# is in the same directory as your SQLite file.:

conn = sqlite3.connect("bookcrossing.sqlite3")


The connection is like hooking up a hose to a pipe without turning on the valve. It prepares a connection, but it doesn't yet make any data flow through the pipe. The variable `conn` is a connection object.

In [4]:
conn

We can now pass a SQL query to the database through this connection. The result we get back is commonly called a "cursor." It's not directly a list of results, but an object through which we can get back one or more results, as well as other information about the running of the query.

In [5]:
cursor = conn.execute("SELECT * FROM books LIMIT 10;")

To actually get back all the results, let's call the `fetchall` method. There are other methods, like `fetchone`, that give us options for how much data we ask for at once.

In [6]:
results = cursor.fetchall()

In [7]:
len(results)

10

In [8]:
results[0]

('0195153448',
 'Classical Mythology',
 'Mark P. O. Morford',
 2002,
 'Oxford University Press')

The column names are available through the cursor object.

In [9]:
cursor.description

(('ISBN', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('author', None, None, None, None, None, None),
 ('year', None, None, None, None, None, None),
 ('publisher', None, None, None, None, None, None))

We can use a Python "list comprehension" (a kind of in-line loop that iterates over one sequence to build up a new list) to make a simple list of column names.

In [10]:
columns = [c[0] for c in cursor.description]
columns

['ISBN', 'title', 'author', 'year', 'publisher']

There are various ways to format strings in Python. The first print statement below uses `f-strings`, which provide a convenient way to format data within text strings with a kind of template structure marked by curly braces.

In [11]:
for result in results:
    for (i, field) in enumerate(result):
        print(f"{columns[i]:>20s}:  {field}")
    print("-" * 60)

                ISBN:  0195153448
               title:  Classical Mythology
              author:  Mark P. O. Morford
                year:  2002
           publisher:  Oxford University Press
------------------------------------------------------------
                ISBN:  0002005018
               title:  Clara Callan
              author:  Richard Bruce Wright
                year:  2001
           publisher:  HarperFlamingo Canada
------------------------------------------------------------
                ISBN:  0060973129
               title:  Decision in Normandy
              author:  Carlo D'Este
                year:  1991
           publisher:  HarperPerennial
------------------------------------------------------------
                ISBN:  0374157065
               title:  Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It
              author:  Gina Bari Kolata
                year:  1999
           publisher:  Farrar S

<div style="background: #CCFFCC; padding: 0.5em"><h1>Exercise 1</h1>
   The cell below contains a copy of the cell above. Modify it so that instead of 20 spaces, it gives 30 spaces to the field names, and modify the end-of-record divider so that it prints 80 hyphens instead of 60. Then run the cell and compare outputs.
    </div>

In [15]:
# EXERCISE 1
for result in results:
    for (i, field) in enumerate(result):
        print(f"{columns[i]:>20s}:  {field}")
    print("-" * 60)

                ISBN:  0195153448
               title:  Classical Mythology
              author:  Mark P. O. Morford
                year:  2002
           publisher:  Oxford University Press
------------------------------------------------------------
                ISBN:  0002005018
               title:  Clara Callan
              author:  Richard Bruce Wright
                year:  2001
           publisher:  HarperFlamingo Canada
------------------------------------------------------------
                ISBN:  0060973129
               title:  Decision in Normandy
              author:  Carlo D'Este
                year:  1991
           publisher:  HarperPerennial
------------------------------------------------------------
                ISBN:  0374157065
               title:  Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It
              author:  Gina Bari Kolata
                year:  1999
           publisher:  Farrar S

<hr/>

# Getting a table of all the data

Now let's get all the data into one big Python list, using SQL to do the join. 

If you don't already have the `crossingdata` view in your SQLite database, the next cell will create it. The triple quotation marks allow us to have a multi-line string in Python.

In [16]:
cursor = conn.execute("""CREATE VIEW IF NOT EXISTS crossingdata AS 
SELECT books.ISBN, title, author, year, publisher, users.userid, rating, location
FROM books JOIN ratings ON books.ISBN = ratings.ISBN 
           JOIN users ON users.userid = ratings.userid;""")

Now we can get all that data from the view.

In [17]:
cursor = conn.execute("SELECT * FROM crossingdata;")
results = cursor.fetchall()

Now we have results as a regular Python list of tuples of strings. We can take the length of the list, index into the list, and iterate over the list in the usual ways.

In [18]:
len(results)

443586

In [19]:
results[0]

('0195153448',
 'Classical Mythology',
 'Mark P. O. Morford',
 2002,
 'Oxford University Press',
 2,
 0,
 'stockton, california, usa')

We could use a "for" loop to build up a list by appending to a variable, but there are other options.

The syntax below is called a `list comprehension`, and in Python it's a typical way of building up a list by iterating over an existing list. 

Since location is the last column, or the last value in the tuple, when `result` is a tuple like the one above, `result[-1]` gives us the last value, which is the location.

In [20]:
locations = [result[-1] for result in results]

Here are the first five:

In [21]:
locations[:5]

['stockton, california, usa',
 'timmins, ontario, canada',
 'ottawa, ontario, canada',
 'comber, ontario, canada',
 'guelph, ontario, canada']

<div style="background: #CCFFCC; padding: 0.5em"><h1>Exercise 2</h1>
   Modify the list comprehension that we used for locations to get a similar list of publishers.
    </div>

In [37]:
# Exercise 2
publishers = _

<hr/> Let's say we want to list and count just the countries. How can we extract countries from the location strings? 

This is the sort of job where string manipulation in Python can be much more convenient than string manipulation in SQL. 

Before we extract the country part of the string from hundreds of thousands of instances, let's work with just one string to figure out what we want to do.

In [38]:
loc = locations[0]

In [39]:
loc

'stockton, california, usa'

In [40]:
loc.split(",")

['stockton', ' california', ' usa']

In [41]:
loc.split(",")[-1]

' usa'

In [42]:
loc.split(",")[-1].strip()

'usa'

Now we can run this on all the elements of the list by putting in a list comprehension that builds up a new list:

In [43]:
countries = [loc.split(",")[-1].strip() for loc in locations]

In [44]:
len(countries)

443586

In [45]:
locations[:10]

['stockton, california, usa',
 'timmins, ontario, canada',
 'ottawa, ontario, canada',
 'comber, ontario, canada',
 'guelph, ontario, canada',
 'halifax, nova scotia, canada',
 'fredericton, new brunswick, canada',
 'timmins, ontario, canada',
 'wichita, kansas, usa',
 'timmins, ontario, canada']

In [46]:
countries[:10]

['usa',
 'canada',
 'canada',
 'canada',
 'canada',
 'canada',
 'canada',
 'canada',
 'usa',
 'canada']

We could write our own "for" loop and build up a dictionary to count how many instances of each value there are, but this is a common thing to want to do, and Python has a convenient library that makes this easy. The `collections` utility has many useful tools, but in this case we just need to import one of them:

In [47]:
from collections import Counter

In [48]:
countrycount = Counter(countries)

In [49]:
len(countrycount.most_common())

175

## Pandas and Dataframes

The Pandas library is a popular library for working with tabular data natively in Python. It offers objects called dataframes to manage and interact with tabular data. These have some functional similarities to objects called dataframes in the R language, which comes out of statistical computing. 

It's conventional to assign the Pandas library the alias `pd` when importing it, which saves a little typing over time.

In [50]:
import pandas as pd

# Reading directly from a database into Pandas

In [51]:
#conn = sqlite3.connect("/content/drive/My Drive/iph430/bookcrossing.sqlite3")
conn = sqlite3.connect("bookcrossing.sqlite3")

Pandas' `read_sql_query` method takes SQL query and a database connection and parses values into a DataFrame, which is like a programmable spreadsheet.

In [52]:
books = pd.read_sql_query("SELECT * FROM books;", conn)

In [53]:
type(books)

pandas.core.frame.DataFrame

The DataFrame's `head` method lets us look at the first several rows of data. Similarly `tail` will look at the last several rows.

In [54]:
# compare LIMIT 5 (as a default) in SQL
books.head()

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company


In [55]:
# compare LIMIT 10 in SQL:
books.head(10)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner


In [56]:
# the number of rows
len(books)

271379

In [57]:
# a tuple of the number of rows and the number of columns
books.shape

(271379, 5)

In [58]:
# Compare selecting only certain columns in SQL:
books[['title','author']].head()

,title,author
0,Classical Mythology,Mark P. O. Morford
1,Clara Callan,Richard Bruce Wright
2,Decision in Normandy,Carlo D'Este
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,The Mummies of Urumchi,E. J. W. Barber


We can also aggregate over values and calculate counts or averages.

In [59]:
# Compare GROUP BY in SQL:
# The renaming always strikes me as a little clunky, because without it the count column 
# is confusingly labeled "ISBN"
books[['ISBN','year']].groupby("year").count().rename(columns={"ISBN":"count"})

,count
year,
0,4619
1376,1
1378,1
1806,1
1897,1
...,...
2026,1
2030,7
2037,1


In [60]:
# Compare ORDER BY in SQL:
books[['ISBN','year']].groupby("year").count().rename(columns={"ISBN":"count"}).sort_values(by="count", ascending=False)

,count
year,
2002,17628
1999,17432
2001,17360
2000,17235
1998,15767
...,...
1906,1
1378,1
1806,1


In [61]:
type(books[['year']])

pandas.core.frame.DataFrame

In [62]:
# Selecting one column returns a Pandas Series, which is a kind of labeled ordered list of values; 
# it's like a one-column DataFrame, but it's not a DataFrame, because it can only be one column.

books['year']

0         2002
1         2001
2         1991
3         1999
4         1999
          ... 
271374    1988
271375    1991
271376    2004
271377    1996
271378    2000
Name: year, Length: 271379, dtype: int64

In [63]:
# We could get an equivalent one-column DataFrame by doubling the brackets:

books[['year']]

,year
0,2002
1,2001
2,1991
3,1999
4,1999
...,...
271374,1988
271375,1991
271376,2004
271377,1996


In [64]:
books['year'] == 1994

0         False
1         False
2         False
3         False
4         False
          ...  
271374    False
271375    False
271376    False
271377    False
271378    False
Name: year, Length: 271379, dtype: bool

In [65]:
# A series compared with a value effectively compares each value in the series with that value, and returns a series of booleans.

books['year'] == 1994

0         False
1         False
2         False
3         False
4         False
          ...  
271374    False
271375    False
271376    False
271377    False
271378    False
Name: year, Length: 271379, dtype: bool

In [66]:
# A series of booleans as long as the number of rows in a dataframe can be used as a filtering index for that
# dataframe; rows indexed with a False value are dropped.
# Compare this to a WHERE clause in SQL

books1999 = books[books['year'] == 1999]

In [67]:
books1999.head()

,ISBN,title,author,year,publisher
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company
13,1552041778,Jane Doe,R. J. Kaiser,1999,Mira Books
16,1575663937,More Cunning Than Man: A Social History of Rat...,Robert Hendrickson,1999,Kensington Publishing Corp.
18,0440234743,The Testament,John Grisham,1999,Dell


In [68]:
# Pandas also has a query method that offers its own query language that can resemble
# a WHERE clause more closely:

books.query("year == 1991")

,ISBN,title,author,year,publisher
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
34,3442410665,Sturmzeit. Roman.,Charlotte Link,1991,Goldmann
61,0679810307,Shabanu: Daughter of the Wind (Border Trilogy),SUZANNE FISHER STAPLES,1991,Laurel Leaf
76,1558531025,Life's Little Instruction Book (Life's Little ...,H. Jackson Brown,1991,Thomas Nelson
...,...,...,...,...,...
271142,0312926715,Ava's Men,Jane Ellen Wayne,1991,St. Martin's Press
271165,0806512156,New York Sports Quiz,Brenda Alesii,1991,Carol Publishing Corporation
271336,359610601X,Der gelbe Stern: Die Judenvernichtung in Europ...,Gerhard Schoenberner,1991,Fischer
271344,0747204020,Illustrated Encyclopedia of Cacti,Glass Charles,1991,Headline


### Joining data

All of the above has shown some things that can be done with one table or view. We can also join multiple tables in Pandas without reference to SQL joins. The syntax is different from SQL, but the underlying logic is very similar.

First let's load the other two tables into dataframes:

In [69]:
ratings = pd.read_sql_query("SELECT * FROM ratings;", conn)
ratings.head()

,userid,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


<div style="background: #CCFFCC; padding: 0.5em"><h1>Exercise 3</h1>
   Following the model of the cell above where we load the ratings table above, write your own code to load the users table into a new variable called `users`, and show the top 5 rows in the same way as above.
    </div>

In [80]:
users = _

<hr/>
A dataframe has a `merge` method that can join it with another dataframe based on values in particular columns, or based on index values. The syntax details can get more complicated than this, but here it's straightforward because we want to join on columns that have the same name in both frames:

In [81]:
users.head()

,userid,location,age
0,1,"nyc, new york, usa",NULL
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",NULL
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",NULL


In [82]:
ratings.merge(users, on="userid").head()

,userid,ISBN,rating,location,age
0,276725,034545104X,0,"tyler, texas, usa",NULL
1,276726,0155061224,5,"seattle, washington, usa",NULL
2,276727,0446520802,0,"h, new south wales, australia",16
3,276729,052165615X,3,"rijeka, n/a, croatia",16
4,276729,0521795028,6,"rijeka, n/a, croatia",16


Because the result of that merge is a dataframe, that resulting dataframe also has a merge method, so we can chain the methods. Compare this with the SQL syntax that we used to join these same tables to make the `crossingdata` view.

In [83]:
crossings = ratings.merge(users, on="userid").merge(books, on="ISBN")

In [84]:
crossings.shape

(443586, 9)

In [85]:
crossings.head()

,userid,ISBN,rating,location,age,title,author,year,publisher
0,276725,034545104X,0,"tyler, texas, usa",NULL,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,"cincinnati, ohio, usa",23,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,163202,034545104X,0,"los angeles, california, usa",26,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,168816,034545104X,0,"santa clara, california, usa",35,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,171912,034545104X,5,"springfield, missouri, usa",NULL,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


In [86]:
crossings.tail(10)

,userid,ISBN,rating,location,age,title,author,year,publisher
443576,276688,0688033628,0,"fort lee, new jersey, usa",NULL,Thursday the Rabbi Walked Out,Harry Kemelman,1978,Harpercollins
443577,276688,0786011157,7,"fort lee, new jersey, usa",NULL,Act of Betrayal,Morgan Avery,2000,Kensington Publishing Corporation
443578,276688,0788192043,0,"fort lee, new jersey, usa",NULL,The Hundred Secret Senses,Amy Tan,2000,DIANE Publishing Company
443579,276688,1575660792,7,"fort lee, new jersey, usa",NULL,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation
443580,276688,1580600506,0,"fort lee, new jersey, usa",NULL,The Fallen Man,Tony Hillerman,1996,HarperCollins
443581,276690,0590907301,0,"wakeman, ohio, usa",43,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple
443582,276704,0380796155,5,"cedar park, texas, usa",NULL,White Abacus,Damien Broderick,1998,Eos
443583,276704,059032120X,0,"cedar park, texas, usa",NULL,Sarah Bishop,Scott Odell,0,Scholastic Inc
443584,276704,0679752714,0,"cedar park, texas, usa",NULL,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA
443585,276704,0806917695,5,"cedar park, texas, usa",NULL,Perplexing Lateral Thinking Puzzles: Scholasti...,Paul Sloane,1997,Sterling Publishing


Now that we have a dataframe keeping our records organized, we have a great way to split out country information into its own column:

In [87]:
crossings['location']

0                    tyler, texas, usa
1                cincinnati, ohio, usa
2         los angeles, california, usa
3         santa clara, california, usa
4           springfield, missouri, usa
                      ...             
443581              wakeman, ohio, usa
443582          cedar park, texas, usa
443583          cedar park, texas, usa
443584          cedar park, texas, usa
443585          cedar park, texas, usa
Name: location, Length: 443586, dtype: object

In [88]:
crossings['location'].map(lambda x: x.split(",")[-1].strip()).head()

0    usa
1    usa
2    usa
3    usa
4    usa
Name: location, dtype: object

In the cells above we just *evaluated* expressions without saving them. Now we can *assign* them to a new column of our existing dataframe:

In [89]:
crossings['country'] = crossings['location'].map(lambda x: x.split(",")[-1].strip())

In [90]:
crossings.head()

,userid,ISBN,rating,location,age,title,author,year,publisher,country
0,276725,034545104X,0,"tyler, texas, usa",NULL,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
1,2313,034545104X,5,"cincinnati, ohio, usa",23,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
2,163202,034545104X,0,"los angeles, california, usa",26,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
3,168816,034545104X,0,"santa clara, california, usa",35,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
4,171912,034545104X,5,"springfield, missouri, usa",NULL,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa


In [91]:
crossings[['country','userid']].groupby("country").count().sort_values(by="userid", ascending=False).head(10)

,userid
country,
usa,322171
canada,46517
united kingdom,13033
germany,11940
,10259
australia,8286
spain,6147
france,3845
portugal,2861


# More complex data reorganization

We can group by the combinations of values in two columns and count up the results. 

Different countries will have different distributions of ages. Let's reorganize the data to let us start to ask questions about those distributions.

First we will group by country and age, and count up. 

After aggregating, country and age make a two-level index. The `reset_index` method flattens everything out so that they look like ordinary columns again.

In [92]:
tmp = crossings[['country','age','userid']].groupby(['country','age']).count()

In [93]:
tmp.columns = ['count']

In [94]:
tmp = tmp.reset_index()

In [95]:
tmp = crossings[['country','age','userid']].groupby(['country','age']).count()
tmp.columns = ['count']
tmp = tmp.reset_index()

<div style="background: #CCFFCC; padding: 0.5em"><h1>Exercise 4</h1>
   The `tmp` dataframe has the data that we're interested in. In the next cell, write <i>and evaluate</i> an expression to look at the first 10 rows of `tmp`.
    </div>

<hr/>Empty values and unexpected values may show up at the extremes in the country column, so rather than look at the head of the dataframe, let's check some values in the middle of it to see what it looks like:

In [96]:
tmp[1001:1012]

,country,age,count
1001,norway,35,1
1002,norway,37,1
1003,norway,45,1
1004,norway,67,1
1005,norway,101,1
1006,norway,NULL,28
1007,nz,220,15
1008,oman,64,1
1009,oman,NULL,1
1010,onondaga nation,NULL,1


This is the kind of data that we're interested in, but it would be convenient to take this long list and make it more rectangular, so that each row is a different country, each column is a different age, and then we could compare counts conveniently either across ages or across countries.

Pandas has pivot tables that are very much like the pivot tables we saw with spreadsheets early in the course.

In [97]:
geo_age = tmp.pivot_table(index="country", columns="age", values="count").fillna(0)

In [98]:
geo_age

age,0,1,2,3,4,5,6,7,8,9,...,199,200,204,220,223,226,228,229,,NULL
country,,,,,,,,,,,,,,,,,,,,,
,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5818.0
"\""n/a\"";""54",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alderney,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vietnam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
we`re global!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
x,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


There are some unusual-looking values in the country names, and some implausibly high numbers in the column labels.

In [ ]:
geo_age.index

In [ ]:
geo_age.columns.values

This will trim the columns to ages between 0 and 84:

In [ ]:
geo_age[list(range(85))]

This will give us a list of countries to subset on:

In [ ]:
countrysubset = geo_age.index[geo_age.sum(axis=1) > 10]
countrysubset

In [ ]:
geo_age.columns[geo_age.sum(axis=0) > 100]

In [ ]:
subset = geo_age.loc[countrysubset, range(84)]
subset

Finally, if we like this organization of the data and want to save it in another format -- let's say we now want to save it to a spreadsheet and share it with someone who can load it in Excel. We can do that directly in Pandas by making up a filename:

In [ ]:
subset.to_excel("geo_age_subset.xlsx")

In [ ]:
conn.close()

If you are using Google Colab, you may want to set the path to a full Google Drive path to save the file indefinitely, or you can download it from the side panel in Jupyter before you end your Colab session. 

Although Colab has local storage space and will create the file without a Google Drive path, the file will be lost when you shut down your Colab session unless you download it first. Colab will save your Jupyter notebook itself, however.